[Amirhossein Mahmoudi](https://ammahmoudi.github.io)
# Adams Variable Step size Predictor Corrector with N step Adams-Bashforth and N-1 step Adams_Moulton Method
solution to [Numerical Analysis (R. Burden, Aires, A. Burden) [10th Ed.]](https://www.amazon.com/Richard-Burden-Numerical-Analysis-Hardcover/dp/B00SB3UL20) Section 5.7 question 8

![5_7_8](images/5_7_8.png)

In [11]:
import pandas as pd
from math import *
import numpy as np

# Corrector Values 

In [12]:
AB_correctors = [
	[2, 3, -1],
	[12, 23, -16, 5],
	[24, 55, -59, 37, -9],
	[720, 1901, -2774, 2616, -1274, 251]
]
AM_correctors=[
    [2, 1, 1],
    [12, 5, 8, -1],
    [24, 9, 19, -5,1],
    [720, 251, 646, -264, 106,-19]
    
]

# RK-4 Method

In [13]:
def rk4(f, t, y,h):
	k1 = h*f(t, y)
	k2 = h*f(t + h/2, y + k1/2)
	k3 = h*f(t + h/2, y + k2/2)
	k4 = h*f(t + h, y + k3)
	t=t+h
	#print(t)
	return t,y + (k1 + 2*k2 + 2*k3 + k4)/6

# Adams Variable Step size Predictor Corrector with N step Adams-Bashforth and N-1 step Adams_Moulton Method

In [14]:
def abN_rk4(f,t, y, b, h_min,h_max,tol,N,AB_correctors,AM_correctors):
	#coeffients
	corrector = AB_correctors[N-2]
	#intialize time array
	time = [t]
	#initialze y_vaules
	y_values = [y]
	result=[]
	flag=True
	last=False
	h=h_max

	for _i in range(1,N):
		t_i,y_i=rk4(f, time[_i-1], y_values[_i-1],h)
		y_values.append(y_i)
		time.append(t_i)
	nflag=True
	i=N
	t=time[len(time)-1]+h
	#run the algorithm
	while(flag):
		fix = 0
	
		for j in range(N):
			fix += corrector[j+1] * f(time[i-j-1], y_values[i-j-1])
		wp = y_values[i-1] + h/corrector[0]*fix

		
		coeffs = AM_correctors[N-2]
		fix = coeffs[1]*f(t, wp)
	
		for j in range(len(coeffs)-2):
			fix += coeffs[j+2] * f(time[i-j-1], y_values[i-j-1])
		wc = y_values[i-1] + h/coeffs[0]*fix

		sigma=19*abs(wc-wp)/(270*h)

		if(sigma<=tol):
			y_values.append(wc)
			time.append(t)
			if nflag:
				for j in range(N):
					result.append([i-N+j,time[i-N+j],y_values[i-N+j],h])
			else:
				result.append([i,time[i],y_values[i],h])
			if last:
				# print('last')
				result.append([i,time[i],y_values[i],h])
				flag=False
			else:
				i=i+1

				nflag=False
				if sigma<=(0.1*tol) or time[i-1]+h>b :
					q=(tol/(2*sigma))**(0.25)
					if q>4:
						h=4*h
						
					else: 
						h=q*h
						
					if h>h_max:
						h=h_max
						
					if time[i-1]+4*h>b:
						h=(b-time[i-1])/4
					
						last=True
					
					for _i in range(-1,N-2):
						t_i,y_i=rk4(f, time[i+_i], y_values[i+_i],h)
						y_values.append(y_i)
						time.append(t_i)
					nflag=True
					i=i+N-1
		else:
			# print('sigma exceed tol')

			q=(tol/(2*sigma))**0.25


			if(q<0.1):h=0.1*h
			else: h=q*h
			

			if h<h_min:
				flag=False
			#	print('h_min exceeded')
			else :
				if nflag:
					i=i-(N-1)

				for _i in range(-1,N-2):
						t_i,y_i=rk4(f, time[i+_i], y_values[i+_i],h)
						if((i+_i+1)>=len(y_values)):
							y_values.append(y_i)
							time.append(t_i)
						    
						else:

							y_values[i+_i+1]=y_i
							time[i+_i+1]=t_i
				nflag=True
				i=i+N-1
		t=time[len(time)-1]+h

	return result


# Test

In [15]:
# f = lambda t, y: y-t**2+1

f = lambda t, y: 1+y/t+(y/t)**2
f_actual = lambda t: t*tan(log(t))


t = 1
y = 0
b = 3
h_max = 0.5
h_min=0.02
tol=1e-6
N=5

result=pd.DataFrame()

result_n=abN_rk4(f,t,y,b,h_min,h_max,tol,N,AB_correctors,AM_correctors)
result_actual=[f_actual(result_n[i][1]) for i in range(len(result_n))]
result=np.c_[result_n,result_actual]
result=pd.DataFrame(result,columns=['i','time','w','h','exact'])
result.head(n=100)

    
    

,i,time,w,h,exact
0,0.0,1.000000,0.000000,0.050000,0.000000
1,1.0,1.050000,0.051270,0.050000,0.051270
2,2.0,1.100000,0.105160,0.050000,0.105160
3,3.0,1.150000,0.161781,0.050000,0.161781
4,4.0,1.200000,0.221243,0.050000,0.221243
5,6.0,1.300000,0.349121,0.050000,0.349121
6,7.0,1.350000,0.417759,0.050000,0.417759
7,8.0,1.400000,0.489682,0.050000,0.489682
8,9.0,1.450000,0.565011,0.050000,0.565011
9,9.0,1.450000,0.565011,0.080090,0.565011
